# Target experiments

- [1] Q 
- [2] Q+A 
- [3] QA + rerank jina
- [4] QA + rerank bert-msmarco
- [5] QA rephrase with llama 3.1 8b
- [6] QA rephrase with gpt 4o mini
- [7] QA + finetune retriever
- [8] QA + finetune retriever + reranker jina
- [9] QA + augmented finetune retriever

# Desired results
- csv with question id, "input", retrieved documents (10 documents)

In [1]:
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder

class VectorStore():
    def __init__(self, embedding_model: SentenceTransformer, rerank_model: CrossEncoder):
        self.embedding_model = embedding_model
        self.rerank_model = rerank_model
        self.vector_store = []
        self.embedding_matrix = None

    def index_documents(self, documents):
        # Encode and normalize all document embeddings
        embeddings = self.embedding_model.encode(
            [doc['payload'] for doc in documents],
            convert_to_tensor=True,
            normalize_embeddings=True
        )

        # Store each document along with its embedding
        self.vector_store = []
        for idx, embedding in enumerate(embeddings):
            self.vector_store.append({
                'id': documents[idx]['id'],
                'payload': documents[idx]['payload'],
                'vector': embedding  # keep it as a torch tensor
            })

        # Stack all embeddings into a single tensor for search
        self.embedding_matrix = torch.stack([entry['vector'] for entry in self.vector_store])

    def search(self, query, top_k=6):
        # Encode and normalize the query
        query_embedding = self.embedding_model.encode(
            [query],
            convert_to_tensor=True,
            normalize_embeddings=True,
            show_progress_bar=False
        )  # shape: (1, dim)

        # Compute cosine similarity (dot product of normalized vectors)
        scores = torch.matmul(query_embedding, self.embedding_matrix.T)  # shape: (1, num_docs)

        # Get top_k scores and corresponding indices
        top_k_scores, top_k_indices = torch.topk(scores, k=top_k, dim=1)

        # Retrieve top_k documents
        top_k_documents = [self.vector_store[idx] for idx in top_k_indices[0].tolist()]

        return top_k_documents, top_k_scores[0].tolist()

    def rerank(self, query, top_k_documents, top_k=6):
        scores = self.rerank_model.rank(
            query=query,
            documents=[doc['payload'] for doc in top_k_documents]
        )
        return [top_k_documents[idx['corpus_id']] for idx in scores][:top_k]


In [2]:
import pandas as pd

df = pd.read_csv("../data/dataset_V3/corpus.csv", index_col="id")
documents = []
for idx, row in df.iterrows():
    documents.append({"id": idx, "payload": f"{row['title_metadata']} | {row['content']}"})

In [3]:
import pandas as pd

def load_merge_df():
    splits = ["1_train", "1_test", "2"]

    specs = [
        # (model_display_name, base_path_with_{split})
        ("o4-mini (best reasoning)", "/Users/vladman/Desktop/university/driving-with-llms/results/qa/qa_strat_6_split_{}.csv"),
        ("Mistral (worst)",     "/Users/vladman/Desktop/university/driving-with-llms/results/qa_vllm/qa_strat_4_split_{}_vllm.csv"),
        ("Gemma (best open)",         "/Users/vladman/Desktop/university/driving-with-llms/results/qa_vllm2/qa_strat_4_split_{}_vllm.csv"),
    ]

    frames = []

    for model_name, base in specs:
        for split in splits:
            base_path = base.format(split)
            df = pd.read_csv(base_path)

            sub = df[["id", "exact_match", "output_prompt"]].copy()
            sub.columns = ["id", "exact_match", "output_prompt"]
            sub.insert(0, "model", model_name)
            sub.insert(1, "split", split)

            frames.append(sub)

    agg = pd.concat(frames, ignore_index=True)
    return agg

df = load_merge_df()
# Group by question ID and check if all models got it wrong
wrong_ids = (
    df.groupby("id")["exact_match"]
    .apply(lambda x: (x == False).all())  # or: (~x).all()
)

# Filter the dataframe to only those IDs where all models were wrong
df_all_wrong = df[df["id"].isin(wrong_ids[wrong_ids].index)]

wrong_ids = df_all_wrong["id"].unique()
print(len(wrong_ids))

46


In [10]:
import pandas as pd
import ast
from tqdm import tqdm
import pytrec_eval
from collections import defaultdict
import matplotlib.pyplot as plt
from collections.abc import Callable, Awaitable
import numpy as np

def eval_ir_framework(dataset: pd.DataFrame, strategy: Callable, name: str, top_k_search=10):
    print('='*30)
    print(f"{name}")
    print('='*30)

    fnqrel = {}
    fqrel = {}
    frun = defaultdict(dict)

    docs_retrieved = []

    for idx, item in tqdm(dataset.iterrows()):
        if item["id"] not in wrong_ids:
            continue
        
        articles = ast.literal_eval(item['legislation'])  # ground truth: list of relevant articles
        question = item["question"]  # the question for search
        possible_answer = " | ".join([entry['answer_text'] 
                                    for entry in ast.literal_eval(item['answers'])])
        relevant_set = list(set(articles))

        retrieved_set_1 = strategy(question, possible_answer, item['id'])
        docs_retrieved.append(retrieved_set_1)
        qrel = {}
        for doc in relevant_set:
            qrel[doc] = 1
        
        nqrel = {}
        for doc in relevant_set:
            nqrel[doc] = 2

        fqrel[f'{idx}-query'] = qrel
        fnqrel[f'{idx}-query'] = nqrel

        for i in range(1, top_k_search+1):
            run = {}
            for doc, score in zip(retrieved_set_1[:i], [1 - 0.05 * x for x in range(10)]):
                run[doc] = score
            frun[str(i)][f'{idx}-query'] = run
        
        
    recalls = []
    precisions = []
    ndcgs = []
    for i in range(1, top_k_search+1):
        evaluator = pytrec_eval.RelevanceEvaluator(fqrel, {f'recall.{i}'})
        results = evaluator.evaluate(frun[str(i)])
        avg_recall = np.mean([res[f'recall_{i}'] for res in list(results.values())])
        # print(f"Avg Recall@{i}: {avg_recall}")
        recalls.append(avg_recall.item())

        evaluator = pytrec_eval.RelevanceEvaluator(fqrel, {f'P.{i}'})
        results = evaluator.evaluate(frun[str(i)])
        avg_precision = np.mean([res[f'P_{i}'] for res in list(results.values())])
        # print(f"Avg Precision@{i}: {avg_precision}")
        precisions.append(avg_precision.item())

        evaluator = pytrec_eval.RelevanceEvaluator(fnqrel, {f'ndcg_cut.{i}', })
        results = evaluator.evaluate(frun[str(i)])
        avg_ndcg = np.mean([res[f'ndcg_cut_{i}'] for res in list(results.values())])
        # print(f"Avg NDCG@{i}: {avg_ndcg}")
        ndcgs.append(avg_ndcg.item())
    

    # dataset['retrieved_documents'] = docs_retrieved

    return recalls, precisions, ndcgs, None

    


In [ ]:
vector_store_simple = VectorStore(SentenceTransformer('intfloat/multilingual-e5-small') , None)
vector_store_simple.index_documents(documents)

In [5]:
vector_store_finetuned = VectorStore(SentenceTransformer('../models/data_trained_V2') , None)
vector_store_finetuned.index_documents(documents)

In [ ]:
vector_store_finetuned_jina = VectorStore(SentenceTransformer('../models/data_trained_V2') , CrossEncoder("jinaai/jina-reranker-v2-base-multilingual",
                            automodel_args={"torch_dtype": "auto"},
                            trust_remote_code=True))
vector_store_finetuned_jina.index_documents(documents)

In [ ]:
vector_store_jina = VectorStore(SentenceTransformer('intfloat/multilingual-e5-small') , CrossEncoder("jinaai/jina-reranker-v2-base-multilingual",
                            automodel_args={"torch_dtype": "auto"},
                            trust_remote_code=True))
vector_store_jina.index_documents(documents)

In [ ]:
vector_store_bertms = VectorStore(SentenceTransformer('intfloat/multilingual-e5-small') , CrossEncoder("amberoad/bert-multilingual-passage-reranking-msmarco"))
vector_store_bertms.index_documents(documents)

In [ ]:
vector_store_finetuned_augmented = VectorStore(SentenceTransformer('../models/augmented_V2') , None)
vector_store_finetuned_augmented.index_documents(documents)

In [6]:
df_gpt_rephrased = pd.read_csv('../results/results-ir-question-rephrase-gpt-4o-mini.csv')

def strategy_1(question, possible_answer, id):
    top_k_documents, top_k_scores = vector_store_simple.search(f"{question}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_2(question, possible_answer, id):
    top_k_documents, top_k_scores = vector_store_simple.search(f"{question} | {possible_answer}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_3(question, possible_answer, id):
    top_k_documents, top_k_scores = vector_store_jina.search(f"{question} | {possible_answer}", top_k=40)
    top_k_documents = vector_store_jina.rerank(f"{question} | {possible_answer}", top_k_documents, top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_4(question, possible_answer, id):
    top_k_documents, top_k_scores = vector_store_bertms.search(f"{question} | {possible_answer}", top_k=40)
    sentences = [
        [f"{question} | {possible_answer}", doc['payload']]
        for doc in top_k_documents]
        
    res = vector_store_bertms.rerank_model.predict(sentences)
    relevant_scores = [score[1] for score in res]
    doc_score_pairs = list(zip(top_k_documents, relevant_scores))
    doc_score_pairs.sort(key=lambda x: x[1], reverse=True)
    top_docs = doc_score_pairs[:10]
    retrieved_set = [doc['id'] for doc, score in top_docs]
    return retrieved_set

def strategy_6(question, possible_answer, id):
    rephrased_question = df_gpt_rephrased.loc[df_gpt_rephrased['id'] == id, 'rephrased_query'].values[0]
    top_k_documents, top_k_scores = vector_store_simple.search(f"{rephrased_question} | {possible_answer}", top_k=40)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_7(question, possible_answer, id):
    top_k_documents, top_k_scores = vector_store_finetuned.search(f"{question} | {possible_answer}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_8(question, possible_answer, id):
    top_k_documents, top_k_scores = vector_store_finetuned_jina.search(f"{question} | {possible_answer}", top_k=40)
    top_k_documents = vector_store_finetuned_jina.rerank(f"{question} | {possible_answer}", top_k_documents, top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set

def strategy_9(question, possible_answer, id):
    top_k_documents, top_k_scores = vector_store_finetuned_augmented.search(f"{question} | {possible_answer}", top_k=10)
    retrieved_set = [doc['id'] for doc in top_k_documents]
    return retrieved_set


# All Experiments

In [ ]:
df_split_1_train = pd.read_csv('../data/dataset_V3/split_1_train.csv')
df_split_1_test = pd.read_csv('../data/dataset_V3/split_1_test.csv')
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_train, strategy_1, '1-train')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_1_train.csv', index=False)
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_test, strategy_1, '1-test')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_1_test.csv', index=False)
##########################################



In [ ]:
df_split_1_train = pd.read_csv('../data/dataset_V3/split_1_train.csv')
df_split_1_test = pd.read_csv('../data/dataset_V3/split_1_test.csv')
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_train, strategy_2, '2-train')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_2_train.csv', index=False)
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_test, strategy_2, '2-test')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_2_test.csv', index=False)
##########################################

In [ ]:
df_split_1_train = pd.read_csv('../data/dataset_V3/split_1_train.csv')
df_split_1_test = pd.read_csv('../data/dataset_V3/split_1_test.csv')
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_train, strategy_3, '3-train')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_3_train.csv', index=False)
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_test, strategy_3, '3-test')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_3_test.csv', index=False)
##########################################

In [ ]:
df_split_1_train = pd.read_csv('../data/dataset_V3/split_1_train.csv')
df_split_1_test = pd.read_csv('../data/dataset_V3/split_1_test.csv')
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_train, strategy_4, '4-train')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_4_train.csv', index=False)
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_test, strategy_4, '4-test')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_4_test.csv', index=False)
##########################################

In [ ]:
df_split_1_train = pd.read_csv('../data/dataset_V3/split_1_train.csv')
df_split_1_test = pd.read_csv('../data/dataset_V3/split_1_test.csv')
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_train, strategy_6, '6-train')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_6_train.csv', index=False)
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_test, strategy_6, '6-test')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_6_test.csv', index=False)
##########################################

In [12]:
df_split_1_train = pd.read_csv('../data/dataset_V3/split_1_train.csv')
df_split_1_test = pd.read_csv('../data/dataset_V3/split_1_test.csv')
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_train, strategy_7, '7-train')
print(recalls[9])
# retrieved.to_csv('../results/ir_strat_7_b_train.csv', index=False)
recalls, _, _, _ = eval_ir_framework(df_split_1_test, strategy_7, '7-test')
print(recalls[9])
# retrieved.to_csv('../results/ir_strat_7_b_test.csv', index=False)
##########################################

7-train


510it [00:00, 1485.44it/s]


1.0
7-test


128it [00:00, 4410.63it/s]

0.8333333333333334


In [ ]:
df_split_1_train = pd.read_csv('../data/dataset_V3/split_1_train.csv')
df_split_1_test = pd.read_csv('../data/dataset_V3/split_1_test.csv')
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_train, strategy_8, '8-train')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_8_train.csv', index=False)
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_test, strategy_8, '8-test')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_8_test.csv', index=False)
##########################################

In [ ]:
df_split_1_train = pd.read_csv('../data/dataset_V3/split_1_train.csv')
df_split_1_test = pd.read_csv('../data/dataset_V3/split_1_test.csv')
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_train, strategy_9, '9-train')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_9_train.csv', index=False)
recalls, precisions, ndcgs, retrieved = eval_ir_framework(df_split_1_test, strategy_9, '9-test')
print(recalls)
print(precisions)
print(ndcgs)
retrieved.to_csv('../results/ir_strat_9_test.csv', index=False)
##########################################

In [ ]:
# df_split_1 = pd.read_csv('../data/dataset_V3/split_1.csv')
# seed = 42
# np.random.seed(seed)
# shuffled_df = df_split_1.sample(frac=1, random_state=seed).reset_index(drop=True)
# train_size = int(0.8 * len(shuffled_df))
# df_split_1_train = shuffled_df[:train_size]
# df_split_1_test = shuffled_df[train_size:]


# df_split_1_train.to_csv('../data/dataset_V3/split_1_train.csv', index=False)
# df_split_1_test.to_csv('../data/dataset_V3/split_1_test.csv', index=False)

# recalls, precisions, ndcgs = eval_ir_framework(df_split_1_train, strategy_2, '2-train')
# print(recalls)
# print(precisions)
# print(ndcgs)

# recalls, precisions, ndcgs = eval_ir_framework(df_split_1_test, strategy_2, '2-test')
# print(recalls)
# print(precisions)
# print(ndcgs)

### Stats

In [ ]:
import matplotlib.pyplot as plt

# Count the occurrences of each category
category_counts = df_split_1_train['question_category_id'].value_counts().sort_index()

# Display counts
print(category_counts)

# Plot the distribution as a bar chart
category_counts.plot(kind='bar')
plt.xlabel('Question Category ID')
plt.ylabel('Count')
plt.title('Distribution of Question Categories')
plt.show()


In [ ]:
category_counts = df_split_1_test['question_category_id'].value_counts().sort_index()

# Display counts
print(category_counts)

# Plot the distribution as a bar chart
category_counts.plot(kind='bar')
plt.xlabel('Question Category ID')
plt.ylabel('Count')
plt.title('Distribution of Question Categories')
plt.show()

### Generate queries using LLM - save for later use

In [ ]:
prompt_ir = """Esti un politist rutier. Vorbesti doar Limba romana.
Primesti o grila de la un test auto, alaturi de raspunsurile posibile.
Scopul tau este sa generezi o singura intrebare astfel incat sa poti cauta legile care fac referinta la intrebarea primita.

Raspunsul tau se va incheia cu:

"Raspuns final: [intrebare generata tip string]"

Aceasta este intrebarea:
{question}

Aceastea sunt variantele de raspuns:
{answers}
===================
"""


import ast
from tqdm import tqdm
import re
import base64
from langchain_core.messages import HumanMessage
import asyncio
from langchain_openai import ChatOpenAI


llm_gepeto = ChatOpenAI(model_name="gpt-4o-mini", api_key="", seed=25)

MAX_CONCURRENT_REQUESTS = 45
semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

async def process_item(full_prompt):
    message = HumanMessage(
        content=[
            {"type": "text", "text": full_prompt},
        ],
    )

    async with semaphore:
        res = await llm_gepeto.ainvoke([message])
        res = res.content
    
    return res

async def mass_runner(data: pd.DataFrame):
    tasks = []
    for real_idx, (idx, item) in tqdm(enumerate(data.iterrows())):
        # Create tasks for parallel processing
        answers = " | ".join([entry['answer_text'] 
                                    for entry in ast.literal_eval(item['answers'])])
        tasks.append(process_item(prompt_ir.format(question=item['question'], answers=answers)))
    
    results = await asyncio.gather(*tasks)
    final = []
    for result, (idx, item) in zip(results, data.iterrows()):
        result = result.split('final:')[-1].strip()
        final.append(result)

    data['rephrased_query'] = final

    final_df = data[["id","rephrased_query"]]
    return final_df

df1 = pd.read_csv('../data/dataset_V3/split_1.csv')

results = await mass_runner(df1)
results.to_csv('../results/results-ir-question-rephrase-gpt-4o-mini.csv')
print(results)

In [ ]:
from langchain_ollama import ChatOllama

llm_ollama = ChatOllama(
    model = "llama3.1:8b-instruct-q8_0",
    temperature = 0,
    seed=25
)

def process_item_ollama(full_prompt):
    message = [('human', full_prompt)]

    res = llm_ollama.invoke(message)
    return res

res = process_item_ollama(""""Esti un politist rutier. Vorbesti doar in limba romana.
Trebuie sa rezolvi o grila de la un test auto. Grila poate avea unul sau mai multe raspunsuri corecte. Vei folosi strict legile din Romania.

Gandeste logic, dar nu extrapola peste informatiile oferite. Judeca doar momentul descris, nu presupune alte situatii.

Reguli de gandire:
1. Citeste cu maxima atentie intrebarea si variantele de raspuns.
2. Identifica strict ce prevederi din legislatia rutiera din Romania se aplica situatiei date.
3. Daca raspunsul pare ""mai sigur"" dar este contrar legislatiei, urmeaza legea, nu instinctul de precautie.
4. Alege DOAR raspunsurile care sunt complet corecte conform textului legii — nu ghici, nu completa informatii lipsa.
5. Daca un raspuns corect este mai bun decat altul dat ca si corect, include mai multe situatii specifice sau exceptii, atunci trebuie ales doar acela.
6. Argumenteaza clar de ce ai ales fiecare raspuns corect. Daca exista mai multe raspunsuri corecte, explica fiecare alegere separat.
7. Fii atent la mici detalii care pot schimba sensul intrebarii sau al raspunsurilor (exista intrebari-capcana).


La final, ultima parte din raspuns trebuie sa fie litera sau literele corecte.
De exemplu, raspunsul tau se va incheia cu:

""Raspuns corect: A""  
sau  
""Raspuns corect: A,B""

Aceasta este intrebarea:
În care dintre situații depășirea este interzisă?

Acestea sunt variantele de raspuns:
A în intersecțiile cu circulație nedirijată și la trecerile pentru pietoni semnalizate; | B în intersecții și la o distanță de 50 m de acestea; | C în zona de acțiune a indicatorului „Limitare de viteză“.

Aceastea sunt legile relevante, dar nu neaparat toate sunt relevante:
[Regulament-118]: Reguli de circulație |  Reguli pentru circulația vehiculelor | Conducătorul de vehicul care efectuează depășirea este obligat: a)   să se asigure că acela care îl urmează sau îl precedă nu a semnalizat intenția începerii unei manevre similare și că poate depăși fără a pune în pericol sau fără a stânjeni circulația din sens opus;    b)   să semnalizeze intenția de efectuare a depășirii;    c)   să păstreze în timpul depășirii o distanță laterală suficienta față de vehiculul depășit;    d)   să reintre pe banda sau în șirul de circulație inițial după ce a semnalizat și s-a asigurat că poate efectua această manevră în condiții de siguranță pentru vehiculul depășit și pentru ceilalți participanți la trafic.   

[Regulament-120]: Reguli de circulație |  Reguli pentru circulația vehiculelor | (1)   Se interzice depășirea vehiculelor: a)   în intersecții cu circulația nedirijată;    b)   în apropierea vârfurilor de rampa, când vizibilitatea este redusă sub 50 m;    c)   în curbe și în orice alte locuri unde vizibilitatea este redusă sub 50 m;    d)   pe pasaje denivelate, pe poduri, sub poduri și în tuneluri. Prin excepție, pot fi depășite în aceste locuri vehiculele cu tracțiune animală, motocicletele fără ataș, mopedele și bicicletele, dacă vizibilitatea asupra drumului este asigurată pe o distanță mai mare de 20 m, iar lățimea drumului este de cel puțin 7 m;    e)   pe trecerile pentru pietoni semnalizate prin indicatoare și marcaje;    f)   pe trecerile la nivel cu calea ferată curentă și la mai puțin de 50 m înainte de acestea;    g)   în dreptul stației pentru tramvai, atunci când acesta este oprit, iar stația nu este prevăzută cu refugiu pentru pietoni;    h)   în zona de acțiune a indicatorului ""Depășirea interzisă"";    i)   când pentru efectuarea manevrei se încalcă marcajul continuu, simplu sau dublu, care desparte sensurile de mers, iar autovehiculul circulă, chiar și parțial, pe sensul opus, ori se încalcă marcajul care delimitează spațiul de interzicere;    j)   când din sens opus se apropie un alt vehicul, iar conducătorul acestuia este obligat să efectueze manevre de evitare a coliziunii;    k)   pe sectorul de drum unde s-a format o coloana de vehicule în așteptare, dacă prin aceasta se intră pe sensul opus de circulație.    (2)   Se interzice depășirea coloanei oficiale.

[OUG-55]: Reguli de circulație |  Reguli pentru circulația vehiculelor | Intersecțiile sunt: a)   cu circulație nedirijată;    b)   cu circulație dirijată. În aceasta categorie sunt incluse și intersecțiile în care circulația se desfășoară în sens giratoriu.   

[Regulament-119]: Reguli de circulație |  Reguli pentru circulația vehiculelor | Conducătorul de vehicul care urmează să fie depășit este obligat: a)   să nu mărească viteza de deplasare;    b)   să circule cât mai aproape de marginea din dreapta a părții carosabile sau a benzii pe care se deplasează.   

[OUG-51]: Reguli de circulație |  Reguli pentru circulația vehiculelor | Conducătorul unui vehicul care circulă în spatele altuia are obligația de a păstra o distanță suficientă față de acesta, pentru evitarea coliziunii.

[OUG-45]: Reguli de circulație |  Reguli pentru circulația vehiculelor | (1)   Depășirea este manevra prin care un vehicul trece înaintea altui vehicul ori pe lângă un obstacol, aflat pe același sens de circulație, prin schimbarea direcției de mers și ieșirea de pe banda de circulație sau din șirul de vehicule în care s-a aflat inițial. (2)   Conducătorul vehiculului care se angajează în depășire trebuie să se asigure că vehiculul care circulă în fața sau în spatele lui nu a inițiat o asemenea manevră. (3)   Atunci când prin manevra de depășire se trece peste axa care separa sensurile de circulație, conducătorii de vehicule trebuie să se asigure că din sens opus nu se apropie un vehicul și că dispun de spațiu suficient pentru a reintra pe banda inițială, unde au obligația să revină după efectuarea manevrei de depășire. (4)   Nu constituie depășire, în sensul  alin. (1) , situația în care un vehicul circulă mai repede pe una dintre benzi decât vehiculele care circulă pe altă bandă în același sens de circulație. (5)   Depășirea se efectuează numai pe partea stângă a vehiculului depășit. Tramvaiul sau vehiculul al cărui conducător a semnalizat intenția și s-a încadrat corespunzător părăsirii sensului de mers spre stânga se depășește prin partea dreapta. (6)   Tramvaiul aflat în mers poate fi depășit și pe partea stângă atunci când drumul este cu sens unic sau când între șina din dreapta și marginea trotuarului nu există spațiu suficient.

[Regulament-135]: Reguli de circulație |  Reguli pentru circulația vehiculelor | Conducătorul de vehicul este obligat să acorde prioritate de trecere și în următoarele situații: a)   la intersecția nedirijată atunci când pătrunde pe un drum național venind de pe un drum județean, comunal sau local;    b)   la intersecția nedirijată atunci când pătrunde pe un drum județean venind de pe un drum comunal sau local;    c)   la intersecția nedirijată atunci când pătrunde pe un drum comunal venind de pe un drum local;    d)   când urmează să pătrundă într-o intersecție cu circulație în sens giratoriu față de cel care circulă în interiorul acesteia;    e)   când circulă în panta față de cel care urca, dacă pe sensul de mers al celui care urca se află un obstacol imobil. În această situație manevra nu este considerată depășire în sensul prevederilor  art. 120 lit. j) ;    f)   când se pune în mișcare sau la pătrunderea pe drumul public venind de pe o proprietate alăturată acestuia față de vehiculul care circulă pe drumul public, indiferent de direcția de deplasare;    g)   când efectuează un viraj spre stânga sau spre dreapta și se intersectează cu un biciclist care circulă pe o pistă pentru biciclete, semnalizată ca atare;    h)   pietonului care traversează drumul public, prin loc special amenajat, marcat și semnalizat corespunzător ori la culoarea verde a semaforului destinat lui, atunci când acesta se află pe sensul de mers al vehiculului.   

[Regulament-107]: Reguli de circulație |  Reguli pentru circulația vehiculelor | (1)   La intersecțiile prevăzute cu indicatoare și/sau cu marcaje pentru semnalizarea direcției de mers, conducătorii de vehicule trebuie să se încadreze pe benzile corespunzătoare direcției de mers voite cu cel puțin 50 m înainte de intersecție și sunt obligați să respecte semnificația indicatoarelor și marcajelor. (2)   La intersecțiile fără marcaje de delimitare a benzilor, conducătorii vehiculelor ocupa în mers, cu cel puțin 50 m înainte de intersecție, următoarele poziții: a)   rândul de lângă bordura sau acostament, cei care vor să schimbe direcția de mers spre dreapta;    b)   rândul de lângă axa drumului sau de lângă marcajul de separare a sensurilor, cei care vor să schimbe direcția de mers spre stânga. Când circulația se desfășoară pe drumuri cu sens unic, conducătorii de vehicule care intenționează să vireze la stânga sunt obligați să ocupe rândul de lângă bordura sau acostamentul din partea stânga;    c)   oricare dintre rânduri, cei care vor să meargă înainte.    (3)   Dacă în intersecție circulă și tramvaie, iar spațiul dintre șina din dreapta și trotuar nu permite circulația pe două sau mai multe rânduri, toți conducătorii de vehicule, indiferent de direcția de deplasare, vor circula pe un singur rând, lăsând liber traseul tramvaiului. (4)   În cazul în care tramvaiul este oprit într-o stație fără refugiu pentru pietoni, vehiculele trebuie să oprească în ordinea sosirii, în spatele acestuia, și să își reia deplasarea numai după ce ușile tramvaiului au fost închise și s-au asigurat că nu pun în pericol siguranța pietonilor angajați în traversarea drumului public.

[Regulament-114]: Reguli de circulație |  Reguli pentru circulația vehiculelor | (1)   Conducătorii de autovehicule și tramvaie sunt obligați să folosească instalațiile de iluminare și/sau semnalizare a acestora, după cum urmează: ----------- Partea introductivă a alin. (1) al art. 114 a fost modificată de pct. 15 al  , publicată în MONITORUL OFICIAL nr. 454 din 24 iulie 2013. a)   luminile de poziție sau de staționare pe timpul imobilizării vehiculului pe partea carosabilă în afara localităților, de la lăsarea serii și până în zorii zilei, ziua când plouă torențial, ninge abundent sau este ceață densă ori în alte condiții care reduc vizibilitatea pe drumul public;    b)   luminile de întâlnire sau de drum, în mers, atât în localități, cât și în afara acestora, după gradul de iluminare a drumului public;    c)   luminile de întâlnire și cele de ceață pe timp de ceață densă;    d)   luminile de întâlnire ale autovehiculelor care însoțesc coloane militare sau cortegii, transporta grupuri organizate de persoane și cele care tractează alte vehicule sau care transporta mărfuri ori produse periculoase, în timpul zilei;    e)   luminile de întâlnire atunci când plouă torențial, ninge abundent ori în alte condiții care reduc vizibilitatea pe drum;    f)   luminile pentru mersul înapoi atunci când vehiculul este manevrat către înapoi;    g)   luminile indicatoare de direcție pentru semnalizarea schimbării direcției de mers, inclusiv la punerea în mișcare a vehiculului de pe loc.    (2)   Pe timpul nopții, la apropierea a două vehicule care circulă din sensuri opuse, conducătorii acestora sunt obligați ca de la o distanță de cel puțin 200 m să folosească luminile de întâlnire concomitent cu reducerea vitezei. Când conducătorul de autovehicul se apropie de un autovehicul care circulă în față să, acesta este obligat să folosească luminile de întâlnire de la o distanță de cel puțin 100 m. (3)   Pe timpul nopții sau în condiții de vizibilitate redusă conducătorii de autovehicule și tramvaie care se apropie de o intersecție nedirijată prin semnale luminoase sau de către polițiști sunt obligați să semnalizeze prin folosirea alternantă a luminilor de întâlnire cu cele de drum dacă nu încalcă astfel prevederile  alin. (2) . (4)   Pe timpul nopții sau în condiții de vizibilitate redusă autovehiculele sau remorcile cu defecțiuni la sistemul de iluminare și semnalizare luminoasă nu pot fi conduse sau remorcate fără a avea în funcțiune pe partea stânga, în față o lumină de întâlnire și în spate una de poziție. (5)   Luminile de avarie se folosesc în următoarele situații: a)   când vehiculul este imobilizat involuntar pe partea carosabilă;    b)   când vehiculul se deplasează foarte lent și/sau constituie el însuși un pericol pentru ceilalți participanți la trafic;    c)   când autovehiculul sau tramvaiul este remorcat.    (6)   În situațiile prevăzute la  alin. (5) , conducătorii de autovehicule și tramvaie trebuie să pună în funcțiune luminile de avarie, în mod succesiv, în ordinea opririi și în cazul în care această manevră este impusă de blocarea circulației pe sensul de mers. 
				 (7)   Când circulă prin tunel conducătorul de vehicul este obligat să folosească luminile de întâlnire.

[OUG-54]: Reguli de circulație |  Reguli pentru circulația vehiculelor | (1)   Conducătorul de vehicul care executa o manevra de schimbare a direcției de mers, de ieșire dintr-un rând de vehicule staționate sau de intrare într-un asemenea rând, de trecere pe o altă bandă de circulație sau de virare spre dreapta ori spre stânga sau care urmează să efectueze o întoarcere ori să meargă cu spatele este obligat să semnalizeze din timp și să se asigure că o poate face fără să perturbe circulația sau să pună în pericol siguranța celorlalți participanți la trafic. (2)   Semnalizarea schimbării direcției de mers trebuie să fie menținută pe întreaga durată a manevrei.
===================""")
print(res)

In [ ]:
print(res.content)

# other

In [ ]:
res_3 = {
    'recalls': [0.3673763497039359, 0.4715112952181918, 0.5257625516246205, 0.5699899238692343, 0.5947373986167089, 0.6251977907150321, 0.6506232273473653, 0.6653517938000696, 0.6761836343732895, 0.689818754042892],
    'precisions': [0.6990595611285266, 0.4882445141065831, 0.3777429467084639, 0.31739811912225707, 0.2699059561128527, 0.24164054336468127, 0.2165248544558889, 0.19553291536050157, 0.178335074886799, 0.1652037617554859],
    'ndcgs': [0.6990595611285266, 0.6097307413952153, 0.5914752236503356, 0.5974592084870567, 0.6028588794957924, 0.6149046138206468, 0.6242005379014265, 0.6305268473398761, 0.6353666716174842, 0.6410628986256416]
}

In [ ]:
res_4 = {
    'recalls': [0.2903225605811813, 0.4005056724884311, 0.45978317659352136, 0.505028735632184, 0.5360258993879683, 0.5650756331790814, 0.5871921182266009, 0.6097203562720803, 0.6249303378613723, 0.6432166492511319],
    'precisions': [0.5579937304075235, 0.4169278996865204, 0.33176593521421105, 0.28056426332288403, 0.24231974921630092, 0.2173458725182863, 0.19547693685624717, 0.1786833855799373, 0.16440264716126787, 0.15282131661442008],
    'ndcgs': [0.5579937304075235, 0.509001724856592, 0.502477113472332, 0.5105194488495042, 0.5202249975565368, 0.5319279322038794, 0.5403551242672291, 0.5494783435386795, 0.5558664659802995, 0.5628200221958617]
}